In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import torch

from csv import QUOTE_NONE
from datetime import datetime
from pprint import pprint


if 'src' not in sys.path:
    sys.path.append('src')

%reload_ext autoreload
%autoreload 2

from explore.data import DetectorDataProvider, LookUpTable
from explore.graph import IntersectionGraph

In [2]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Data extraction

### Read data from the data set

Get list of all sections

In [ ]:
lut = LookUpTable('Data')
sections = set()
for inter in lut.list_intersections():
    detectors = lut.get_detectors_on(inter)
    for sec in detectors[['Starting Intersection', 'Ending Intersection']].values:
        sections.add(tuple(sorted(sec)))
pprint(list(sections)[:10])

Extract list of detectors for each section

In [ ]:
lut = LookUpTable('Data')
def construct_int_det(sections):
    int_det = []
    for int_1, int_2 in sections:
        det_1_2, det_2_1 = lut.get_detectors_between(int_1, int_2)
        int_det.append((int_1, int_2, det_1_2))
        int_det.append((int_2, int_1, det_2_1))
    int_det = pd.DataFrame(int_det, columns=['Start', 'End', 'Detectors'])
    
    return int_det
    
int_det = construct_int_det(sections)
int_det

Extract data from January to August and accumulate counts by section

In [ ]:
ddp = DetectorDataProvider('Data')
ddp.list_intersections()

In [ ]:
START_DATE = datetime(2021, 1, 1)
END_DATE = datetime(2021, 12, 1)

def count_traffic(int_det, start_date=START_DATE, end_date=END_DATE):
    lens = set()
    def get_count(section_end, detectors):
        section_data = ddp.get_data_for_period(section_end, START_DATE, END_DATE)
        for col in section_data.columns:
            section_data[col] = pd.to_numeric(section_data[col], errors='coerce')
        try:
            return list(section_data[detectors].fillna(0).sum(axis=1, numeric_only=True).cumsum())
        except KeyError:
            print(section_end, section_data.columns, detectors)
    int_det['Counts'] = int_det.apply(lambda sec: get_count(sec['End'], sec['Detectors']), axis=1)

count_traffic(int_det)
int_det.to_hdf('int_det_excluded_missing.hdf', 'int_det')
int_det

### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [ ]:
def construct_q(int_det: pd.DataFrame):
    mat_q = torch.tensor(int_det['Counts'].tolist()).T
    return mat_q

mat_q = construct_q(int_det)
torch.save(mat_q, 'mat_q_excluded_missing.pt')
mat_q.shape

## Data preprocessing

Load matrix $Q$

In [3]:
mat_q = torch.load('mat_q_excluded_missing.pt')

In [4]:
mat_q_fft = torch.abs(torch.fft.fft(mat_q, dim=0))
mat_q_fft.shape

torch.Size([32064, 468])

In [ ]:
plt.plot(mat_q_fft[:, 0])

## Data compression

Construct a correlation coefficient matrix
$$
R(i, j)=\frac{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))(z(s_j,t_k)-\tilde{z}(s_j))}{\sqrt{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))^2}\sqrt{\sum_{k=1}^d(z(s_j,t_k)-\tilde{z}(s_j))^2}},
$$
where $$\tilde{z}(s_i)=\frac{1}{d}\sum_{k=1}^dz(s_i,t_k)$$

In [ ]:
def construct_r(mat_q, ignore: set=None):
    if ignore is None:
        ignore = set()
    d, p = mat_q.shape
    mat_q_normalized = mat_q - mat_q.mean(dim=0)
    mat_r = torch.zeros(p, p)
    for i in range(p):
        for j in range(i, p):
            if (i, j) in ignore:
                mat_r[i][j] = 0.0
            else:
                i_col = mat_q_normalized[:, i]
                j_col = mat_q_normalized[:, j]
                i_norm = max(torch.norm(i_col), 1e-12)
                j_norm = max(torch.norm(j_col), 1e-12)
                if i_norm == 0 or j_norm == 0:
                    if i_norm == j_norm:
                        mat_r[i][j] = 1
                    else:
                        mat_r[i][j] = 0
                else:
                    mat_r[i][j] = (i_col @ j_col) / i_norm / j_norm
            mat_r[j][i] = mat_r[i][j]
    return mat_r

mat_r = construct_r(mat_q)
torch.save(mat_r, 'mat_r_excluded_missing.pt')
mat_r

Load matrices R and Q

In [ ]:
mat_q = torch.load('mat_q_excluded_missing.pt')
mat_r = torch.load('mat_r_excluded_missing.pt')
mat_r = mat_r / mat_r.max()

In [ ]:
ALPHA = 0.9999

n_timesteps, n_sections = mat_q.shape
n_grouped = 0
groups = []

mat_r_copy = mat_r - torch.diag(mat_r.diag())
while n_grouped < n_sections:
    new_group_idx = torch.nonzero(mat_r_copy > ALPHA)
    if len(new_group_idx) > 0:
        corr = mat_r[new_group_idx[:, 0], new_group_idx[:, 1]]
        new_group_idx = new_group_idx[:, 0].unique()

        n_grouped += len(new_group_idx)
        mat_r_copy[new_group_idx, :] = 0
        mat_r_copy[:, new_group_idx] = 0
        groups.append((new_group_idx, corr.min(), corr.max()))
        if mat_r_copy.max() == 0:
            break
        else:
            mat_r_copy /= mat_r_copy.max()
del mat_r_copy

Analyse grouping

In [ ]:
print(f'Using alpha={ALPHA}, {n_sections} correlated sections were divided ' +
      f'into {len(groups)} groups:')
n_ungrouped = 0
for i, (group, corr_min, corr_max) in enumerate(groups, start=1):
      group_coeffs = mat_r[tuple(group.T), :]
      print(f'Group {i} - {len(group)} sections with correlation coefficients '
          f'{corr_min:.3f} to {corr_max:.3f}')
print(str(n_sections - n_grouped) + " section(s) don't correlate with anything and weren't grouped")


In [ ]:
representatives = torch.stack([g[0] for g, _, _ in groups])
mat_c = mat_q[:, representatives]
assert mat_c.shape == (mat_q.shape[0], len(groups))
torch.save(mat_c, 'mat_c_excluded_missing.pt')
mat_c.shape

In [ ]:
mat_x = torch.linalg.pinv(mat_c) @ mat_q
torch.save(mat_x, 'mat_x_excluded_missing.pt')
mat_x.shape

In [ ]:
print(torch.max((mat_c @ mat_x) - mat_q))